In [ ]:
#| include: false
from nbdev.showdoc import *

This part of the tutorial demonstrates how to use `NumerFrame` to download predictions bought on [NumerBay](http://numerbay.ai/) community marketplace. Currently only the main tournament is supported. Signals support will be added in future.

In [ ]:
from nbdev import show_doc
from numerblox.numerframe import create_numerframe, NumerFrame
from numerblox.download import NumeraiClassicDownloader
from numerblox.key import Key
from numerblox.model import NumerBayCSVs
from numerblox.submission import NumerBaySubmitter, NumeraiClassicSubmitter, NumeraiSignalsSubmitter

/opt/miniconda3/envs/classic_prod/lib/python3.9/site-packages/xgboost/compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
2023-09-04 12:52:05.801492: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-04 12:52:05.970809: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-09-04 12:52:06.662648: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot 

First, we download validation data using `NumeraiClassicDownloader`.

In [ ]:
downloader = NumeraiClassicDownloader("numerframe_edu")
# Path variables
tournament_file = "v4.2/validation_int8.parquet"
tournament_save_path = f"{str(downloader.dir)}/{tournament_file}"
# Download only tournament parquet file
downloader.download_single_dataset(tournament_file,
                                   dest_path=tournament_save_path)

📁 Downloading 'v4.2/validation_int8.parquet' 📁

2023-09-04 12:52:08,550 INFO numerapi.utils: starting download
numerframe_edu/v4.2/validation_int8.parquet: 2.16GB [02:11, 16.4MB/s]                             


Loading in data and initializing a `NumerFrame` takes one line of code. It will automatically recognize the data format such as `.csv` or `.parquet`. You have the option to add metadata, which is stored in the `meta` attribute.

In [ ]:
# Initialize NumerFrame from parquet file path
dataf = create_numerframe(tournament_save_path)

In [ ]:
dataf.head(2)

era   data_type  feature_honoured_observational_balaamite  \
id                                                                             
n000101811a8a843  0575  validation                                         2   
n001e1318d5072ac  0575  validation                                         1   

                  feature_polaroid_vadose_quinze  \
id                                                 
n000101811a8a843                               0   
n001e1318d5072ac                               4   

                  feature_untidy_withdrawn_bargeman  \
id                                                    
n000101811a8a843                                  4   
n001e1318d5072ac                                  2   

                  feature_genuine_kyphotic_trehala  \
id                                                   
n000101811a8a843                                 0   
n001e1318d5072ac                                 2   

                  feature_unenthralled_sportful_schoolhouse  \
id                                                            
n000101811a8a843                                          3   
n001e1318d5072ac                                          1   

                  feature_divulsive_explanatory_ideologue  \
id                                                          
n000101811a8a843                                        0   
n001e1318d5072ac                                        3   

                  feature_ichthyotic_roofed_yeshiva  \
id                                                    
n000101811a8a843                                  4   
n001e1318d5072ac                                  3   

                  feature_waggly_outlandish_carbonisation  ...  \
id                                                         ...   
n000101811a8a843                                        1  ...   
n001e1318d5072ac                                        0  ...   

                  target_bravo_v4_20  target_bravo_v4_60  \
id                                                         
n000101811a8a843                0.50                0.50   
n001e1318d5072ac                0.25                0.25   

                  target_charlie_v4_20  target_charlie_v4_60  \
id                                                             
n000101811a8a843                  0.50                  0.50   
n001e1318d5072ac                  0.25                  0.25   

                  target_delta_v4_20  target_delta_v4_60  target_echo_v4_20  \
id                                                                            
n000101811a8a843                0.50                0.50               0.75   
n001e1318d5072ac                0.25                0.25               0.25   

                  target_echo_v4_60  target_jeremy_v4_20  target_jeremy_v4_60  
id                                                                             
n000101811a8a843                0.5                  0.5                 0.50  
n001e1318d5072ac                0.5                  0.0                 0.25  

[2 rows x 2183 columns]

Next, we create a NumerBayCSVs model object with a list of products we bought and NumerBay credentials. `numerbay_key_path` may be required for products that use client-side encryption. You can learn more about encryption [here](https://docs.numerbay.ai/updates/encryption)

In [ ]:
# !pip install numerbay
nb_model = NumerBayCSVs(data_directory='/app/notebooks/tmp',
                        numerbay_product_full_names=['numerai-predictions-numerbay'],  # change to the full names of products you bought
                        numerbay_username="your_username",  # change to your own username
                        numerbay_password="your_password",  # change to your own password
                        numerbay_key_path="/app/notebooks/tmp/numerbay.json")

Call the `predict` method on the `NumerFrame` to fetch the prediction file from NumerBay. If the file already exists in the `data_directory`, that file will be loaded without re-downloading.

In [ ]:
# preds = nb_model.predict(dataf)

The predictions are concatenated to the `NumerFrame` with column name `prediction_numerai-predictions-numerbay`

In [ ]:
# preds

In this part of the tutorial we have downloaded a prediction file from NumerBay with `NumerFrame`. This makes things easier for post processing such as ensembling and neutralization.

------------------------------------------------------

# NumerBay submission

This part of the tutorial is for sellers who want to upload their predictions to NumerBay to fulfill sale orders. Using `NumerBaySubmitter`, a seller can choose to submit to both Numerai and NumerBay or just NumerBay.

Assume we have some prediction column to upload for the Numerai main tournament, in this case the `prediction` column which simply takes the value of a feature.

In [ ]:
# dataf = create_numerframe(tournament_save_path)

In [ ]:
# dataf['prediction'] = dataf['feature_dichasial_hammier_spawner']

Set `upload_to_numerai` to True (default) if you want to submit to both Numerai and NumerBay, set to False to submit only to NumerBay.

In [ ]:
# nb_submitter = NumerBaySubmitter(tournament_submitter=numerai_submitter, upload_to_numerai=True, numerbay_username="numerbay", numerbay_password="your_password")

Finally, we call the `full_submission` method to perform the submission

In [ ]:
# nb_submitter.full_submission(dataf, file_name='upload-full.csv', model_name='numerbay', numerbay_product_full_name='numerai-predictions-numerbay', cols='prediction')

The process for Signals submission is very similar and is omitted for brevity, just do the following:
- Use Signals NumerFrame
- Change `NumeraiClassicSubmitter` to `NumeraiSignalsSubmitter` for the `tournament_submitter` argument
- When calling `full_submission`, change the `cols` argument to the list of Signals column to submit (e.g. `['bloomberg_ticker', 'signal']`)

------------------------------------------------------

After we are done we can easily clean up our downloaded data with one line of code called from the downloader.

In [ ]:
# Clean up environment
downloader.remove_base_directory()

⚠ Deleting directory for 'NumeraiClassicDownloader' ⚠
Path: '/home/clepelaars/numerblox/nbs/edu_nbs/numerframe_edu'